**このコードでは、年毎のフォルダーで保存されている大気汚染物質の生データを一つのcsvにデータ処理を行いながらまとめるという作業を行う**

前提として、

*  History Data Platform COVID-19 Dataset ,The World Air Quality Project からデータを2019年から2023年まで取得したものをgoogle drive内でフォルダーとして保持していること

が前提条件となる。

**注意事項**

※ 作成者はGoogle drive内で作業をしていると同時に、ゼミというフォルダーの中の公開ソースというフォルダーで作業していることからpathの変更は必要不可欠である

In [1]:
#Google driveに接続
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#必要なライブラリをインポート
import pandas as pd
import os

In [3]:
#データ前処理を行う生データを保存しているフォルダを取得
#　※自分で作成したフォルダパスが異なる場合は変更
working_dir = '/content/drive/MyDrive/ゼミ/公開ソース'
path = f'{working_dir}/data/raw_data'
os.listdir(path)

['2021', '2022_2023', '2019', '2020']

In [4]:
#フォルダーごとに分かれている年ごとにデータフレームでまとめる。
for year in (os.listdir(path)):
  paths = f"{path}/{year}"
  year_text = f"df_{year}"
  for number, files in enumerate(os.listdir(paths)):
    #データフレームを読み込む時点で、必要な列のみを取得する
    df = pd.read_csv(f"{paths}/{files}",header=4,usecols=["Date", "Country","City","Specie","median"])
    if number == 0:
      df_connect = df
    else:
      df_connect = pd.concat([df_connect,df],ignore_index=True)
  exec("{}=df_connect".format(year_text))

In [5]:
#年ごとにまとめたデータフレームを結合
df_all = pd.concat([df_2019,df_2020,df_2021,df_2022_2023],ignore_index=True)

In [6]:
#重複している行を削除
df_all_unduplicated = df_all.drop_duplicates(subset={"Date","Country","City","Specie","median"})
#Dateから年を抽出
df_all_unduplicated ["Year"] = pd.to_datetime(df_all_unduplicated["Date"]).dt.strftime("%Y")
#実験対象外である年のデータを削除
df_all_year = df_all_unduplicated[df_all_unduplicated['Year'] != '2018']

<ipython-input-6-703ddf06eb07>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_unduplicated ["Year"] = pd.to_datetime(df_all_unduplicated["Date"]).dt.strftime("%Y")


In [7]:
#インデックスにDateを指定
df_set_index = df_all_year.set_index("Date")

In [8]:
#日付順に並べ替え
df_sort_date = df_set_index.sort_values('Date')

In [9]:
#データを保存
path_csv = f'{working_dir}/data/combined_data'
df_sort_date.to_csv(f'{path_csv}/all_data.csv')